In [ ]:
import os

Base_dir = os.getcwd()

Data_dir=os.path.join(Base_dir,"Data")
train_dir=os.path.join(Data_dir,"train")
validation_dir=os.path.join(Data_dir,"validation")


#extra
cat_train_dir=os.path.join(train_dir,"Cat")
dog_train_dir=os.path.join(train_dir,"Dog")

cat_val_dir=os.path.join(validation_dir,"Cat")
dog_val_dir=os.path.join(validation_dir,"Dog")


In [16]:
Base_dir = os.getcwd()
print(Base_dir)

c:\Users\VirtualUser\Desktop\NANDANA


In [14]:
def count_images(folder,extensions=(".jpg",".png",".jpeg")):
    return len([
        file for file in os.listdir(folder)
        if file.lower().endswith(extensions)
    ])

print("Training")
print("cat:",count_images(cat_train_dir))
print("dog:",count_images(dog_train_dir))

print("Validation")
print("cat:",count_images(cat_val_dir))
print("dog:",count_images(dog_val_dir))

Training
cat: 9999
dog: 9999
Validation
cat: 2500
dog: 2500


In [43]:
#required method
from torchvision import datasets
from torchvision import transforms

train_transform=transforms.Compose([transforms.Resize((224,224)),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225]
                                    ),
                                    transforms.RandomRotation(degrees=15),
                                    transforms.ToTensor(),
                                    transforms.ColorJitter(brightness=0.2,saturation=0.2,contrast=0.2),
                                    transforms.RandomHorizontalFlip()
                                    ])
val_transform=transforms.Compose([transforms.ToTensor(),
                                  transforms.Normalize(
                                        mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225]),
                                    transforms.Resize((224,224))
                                
                                ])





In [44]:
train_dataset=datasets.ImageFolder(
    root=train_dir,
    transform=train_transform
)

val_dataset=datasets.ImageFolder(
    root=validation_dir,
    transform=val_transform
)

In [45]:



val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),   # fixed crop
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [46]:
print(train_dataset.class_to_idx)
print(val_dataset.class_to_idx)

print("train data=",len(train_dataset))
print("validation data=",len(val_dataset))

{'Cat': 0, 'Dog': 1}
{'Cat': 0, 'Dog': 1}
train data= 19998
validation data= 5000
